<a href="https://colab.research.google.com/github/SinnottKayleigh/B2B-Sales-Algos/blob/main/Lead_Source_Quality_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Lead Source Quality Analysis
- Compare conversion rates by lead source
- Track ROI of different lead generation channels
- Analyze deal velocity by source
- Measure deal size variations by origin
- Monitor source-specific engagement patterns

In [1]:
lead_source_metrics = {
    'Source_Types': [
        'LinkedIn Direct Outreach',
        'LinkedIn Content',
        'Email Campaigns',
        'Webinars',
        'Industry Events',
        'Referrals',
        'Website Inbound',
        'Partner Referrals',
        'Trade Shows',
        'Cold Calls'
    ],

    'Key_Metrics': {
        'Volume_Metrics': [
            'Number of leads generated',
            'Number of qualified leads',
            'Number of meetings scheduled',
            'Number of opportunities created',
            'Number of closed deals'
        ],
        'Financial_Metrics': [
            'Cost per lead',
            'Cost per qualified lead',
            'Cost per opportunity',
            'Cost per acquisition',
            'Total revenue generated',
            'Average deal size'
        ],
        'Time_Metrics': [
            'Lead qualification time',
            'Time to first meeting',
            'Sales cycle duration',
            'Time to response'
        ],
        'Quality_Metrics': [
            'Conversion rate',
            'Engagement score',
            'Lead score',
            'Meeting show-up rate',
            'Proposal acceptance rate'
        ]
    }
}


In [2]:
import pandas as pd
import numpy as np

def calculate_source_performance(data):
    performance_metrics = {
        'conversion_rate': lambda x: (x['closed_deals'] / x['total_leads']) * 100,
        'qualification_rate': lambda x: (x['qualified_leads'] / x['total_leads']) * 100,
        'meeting_rate': lambda x: (x['meetings_held'] / x['qualified_leads']) * 100,

        'roi': lambda x: (x['total_revenue'] - x['total_cost']) / x['total_cost'] * 100,
        'avg_deal_size': lambda x: x['total_revenue'] / x['closed_deals'],
        'cost_per_acquisition': lambda x: x['total_cost'] / x['closed_deals'],

        'avg_sales_cycle': lambda x: x['total_days_to_close'] / x['closed_deals'],
        'avg_qualification_time': lambda x: x['total_qualification_days'] / x['qualified_leads'],

        'avg_engagement_score': lambda x: x['total_engagement_points'] / x['total_leads'],
        'deal_size_index': lambda x: x['avg_deal_size'] / x['global_avg_deal_size']
    }

    return performance_metrics

In [3]:
import plotly.express as px
import plotly.graph_objects as go

def create_source_analysis_dashboard(df):
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Conversion Rates by Source',
            'ROI by Source',
            'Average Sales Cycle',
            'Deal Size Distribution'
        )
    )

    fig.add_trace(
        go.Bar(
            x=df['source'],
            y=df['conversion_rate'],
            name='Conversion Rate'
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Bar(
            x=df['source'],
            y=df['roi'],
            name='ROI'
        ),
        row=1, col=2
    )

    fig.add_trace(
        go.Box(
            x=df['source'],
            y=df['sales_cycle'],
            name='Sales Cycle'
        ),
        row=2, col=1
    )

    fig.add_trace(
        go.Violin(
            x=df['source'],
            y=df['deal_size'],
            name='Deal Size'
        ),
        row=2, col=2
    )

    return fig

In [4]:
def calculate_source_score(source_data):
    weights = {
        'conversion_rate': 0.25,
        'roi': 0.25,
        'avg_deal_size': 0.20,
        'sales_cycle': 0.15,
        'engagement_score': 0.15
    }

    normalized_metrics = {
        metric: (source_data[metric] - min(source_data[metric])) /
                (max(source_data[metric]) - min(source_data[metric]))
        for metric in weights.keys()
    }

    final_score = sum(normalized_metrics[metric] * weight
                     for metric, weight in weights.items())

    return final_score * 100  # Convert to 0-100 scale

In [5]:
def generate_recommendations(source_analysis):
    recommendations = []

    for source in source_analysis.index:
        source_metrics = source_analysis.loc[source]

        if source_metrics['roi'] < source_analysis['roi'].mean():
            recommendations.append({
                'source': source,
                'issue': 'Low ROI',
                'action': 'Review cost structure and targeting criteria'
            })

        if source_metrics['conversion_rate'] < source_analysis['conversion_rate'].mean():
            recommendations.append({
                'source': source,
                'issue': 'Low Conversion',
                'action': 'Analyze lead quality and engagement patterns'
            })

    return pd.DataFrame(recommendations)

In [6]:
def generate_monthly_report(source_data, previous_month):
    report = {
        'summary_metrics': {
            'total_leads': source_data['total_leads'].sum(),
            'total_revenue': source_data['total_revenue'].sum(),
            'avg_conversion': source_data['conversion_rate'].mean(),
            'best_performing_source': source_data['roi'].idxmax()
        },
        'month_over_month': {
            'lead_growth': (source_data['total_leads'].sum() /
                          previous_month['total_leads'].sum() - 1) * 100,
            'revenue_growth': (source_data['total_revenue'].sum() /
                             previous_month['total_revenue'].sum() - 1) * 100
        },
        'recommendations': generate_recommendations(source_data)
    }

    return report

KPI Thresholds

In [8]:
from sklearn.preprocessing import MinMaxScaler
from scipy.optimize import linear_sum_assignment

class LeadSourceKPIThresholds:
    def __init__(self):
        self.thresholds = {
            'Conversion_Metrics': {
                'lead_to_opportunity_rate': {
                    'excellent': 0.25,  # 25% or above
                    'good': 0.15,       # 15-25%
                    'average': 0.10,    # 10-15%
                    'poor': 0.05        # Below 5%
                },
                'opportunity_to_close_rate': {
                    'excellent': 0.30,   # 30% or above
                    'good': 0.20,        # 20-30%
                    'average': 0.15,     # 15-20%
                    'poor': 0.10         # Below 10%
                },
                'meeting_show_rate': {
                    'excellent': 0.90,   # 90% or above
                    'good': 0.80,        # 80-90%
                    'average': 0.70,     # 70-80%
                    'poor': 0.60         # Below 60%
                }
            },
            'Financial_Metrics': {
                'cost_per_lead': {
                    'excellent': 50,     # $50 or below
                    'good': 100,         # $50-100
                    'average': 200,      # $100-200
                    'poor': 300          # Above $300
                },
                'roi_percentage': {
                    'excellent': 500,    # 500% or above
                    'good': 300,         # 300-500%
                    'average': 200,      # 200-300%
                    'poor': 100          # Below 100%
                },
                'average_deal_size': {
                    'excellent': 100000,  # $100k or above
                    'good': 50000,       # $50k-100k
                    'average': 25000,    # $25k-50k
                    'poor': 10000        # Below $10k
                }
            },
            'Time_Metrics': {
                'days_to_qualification': {
                    'excellent': 5,      # 5 days or less
                    'good': 10,          # 5-10 days
                    'average': 15,       # 10-15 days
                    'poor': 20           # Above 20 days
                },
                'sales_cycle_length': {
                    'excellent': 30,     # 30 days or less
                    'good': 60,          # 30-60 days
                    'average': 90,       # 60-90 days
                    'poor': 120          # Above 120 days
                }
            },
            'Quality_Metrics': {
                'lead_score': {
                    'excellent': 80,     # 80 or above
                    'good': 60,          # 60-80
                    'average': 40,       # 40-60
                    'poor': 20           # Below 20
                },
                'engagement_score': {
                    'excellent': 8,      # 8-10
                    'good': 6,           # 6-8
                    'average': 4,        # 4-6
                    'poor': 2            # Below 2
                }
            }
        }

    def evaluate_metric(self, metric_category, metric_name, value):
        thresholds = self.thresholds[metric_category][metric_name]

        if value >= thresholds['excellent']:
            return ('excellent', 4)
        elif value >= thresholds['good']:
            return ('good', 3)
        elif value >= thresholds['average']:
            return ('average', 2)
        else:
            return ('poor', 1)

class LeadSourceOptimizer:
    def __init__(self, budget, resources):
        self.budget = budget
        self.resources = resources
        self.kpi_thresholds = LeadSourceKPIThresholds()

    def calculate_source_efficiency(self, source_data):
        """Calculate efficiency score for each lead source"""
        efficiency_metrics = {
            'cost_efficiency': source_data['revenue'] / source_data['cost'],
            'time_efficiency': source_data['closed_deals'] / source_data['sales_cycle_days'],
            'quality_score': source_data['lead_score'] * source_data['conversion_rate'],
            'resource_utilization': source_data['closed_deals'] / source_data['resource_hours']
        }

        return np.mean(list(efficiency_metrics.values()))

    def optimize_budget_allocation(self, historical_data):
        """Optimize budget allocation across lead sources"""
        sources = historical_data['source'].unique()
        n_sources = len(sources)

        cost_matrix = np.zeros((n_sources, n_sources))
        for i, source in enumerate(sources):
            source_data = historical_data[historical_data['source'] == source]
            efficiency = self.calculate_source_efficiency(source_data)
            cost_matrix[i] = np.linspace(efficiency * 0.5, efficiency * 1.5, n_sources)

        row_ind, col_ind = linear_sum_assignment(cost_matrix, maximize=True)

        allocations = {}
        total_efficiency = cost_matrix[row_ind, col_ind].sum()
        for i, source in enumerate(sources):
            allocations[source] = {
                'budget_allocation': (cost_matrix[i, col_ind[i]] / total_efficiency) * self.budget,
                'expected_efficiency': cost_matrix[i, col_ind[i]]
            }

        return allocations

    def generate_recommendations(self, historical_data, allocations):
        """Generate specific recommendations for each lead source"""
        recommendations = []

        for source in historical_data['source'].unique():
            source_data = historical_data[historical_data['source'] == source]
            allocation = allocations[source]

            current_metrics = {
                'conversion_rate': source_data['conversion_rate'].mean(),
                'cost_per_lead': source_data['cost'].sum() / source_data['total_leads'].sum(),
                'roi': (source_data['revenue'].sum() - source_data['cost'].sum()) / source_data['cost'].sum()
            }

            source_recommendations = {
                'source': source,
                'budget_allocation': allocation['budget_allocation'],
                'recommendations': []
            }

            if current_metrics['conversion_rate'] < self.kpi_thresholds.thresholds['Conversion_Metrics']['lead_to_opportunity_rate']['average']:
                source_recommendations['recommendations'].append({
                    'area': 'Conversion',
                    'action': 'Improve lead qualification process',
                    'expected_impact': 'Increase conversion rate by 20%'
                })

            if current_metrics['cost_per_lead'] > self.kpi_thresholds.thresholds['Financial_Metrics']['cost_per_lead']['average']:
                source_recommendations['recommendations'].append({
                    'area': 'Cost',
                    'action': 'Optimize targeting parameters',
                    'expected_impact': 'Reduce cost per lead by 25%'
                })

            recommendations.append(source_recommendations)

        return recommendations

def run_optimization_analysis(historical_data, budget, resources):
    optimizer = LeadSourceOptimizer(budget=budget, resources=resources)

    allocations = optimizer.optimize_budget_allocation(historical_data)

    recommendations = optimizer.generate_recommendations(historical_data, allocations)

    return {
        'allocations': allocations,
        'recommendations': recommendations,
        'summary': {
            'total_budget': budget,
            'number_of_sources': len(allocations),
            'expected_roi': sum(a['expected_efficiency'] for a in allocations.values()) / len(allocations)
        }
    }

In [9]:
# Example data structure
historical_data = pd.DataFrame({
    'source': ['LinkedIn', 'Email', 'Events', 'Referrals'],
    'total_leads': [100, 150, 75, 50],
    'conversion_rate': [0.15, 0.12, 0.20, 0.25],
    'cost': [5000, 3000, 8000, 2000],
    'revenue': [50000, 40000, 60000, 30000],
    'sales_cycle_days': [45, 30, 60, 20],
    'lead_score': [70, 60, 80, 85],
    'resource_hours': [100, 80, 120, 40],
    'closed_deals': [15, 18, 15, 12]
})

results = run_optimization_analysis(
    historical_data=historical_data,
    budget=50000,
    resources={'sales_hours': 500, 'marketing_hours': 300}
)

print("Optimized Budget Allocation:")
for source, allocation in results['allocations'].items():
    print(f"{source}: ${allocation['budget_allocation']:,.2f}")

print("\nRecommendations:")
for rec in results['recommendations']:
    print(f"\n{rec['source']}:")
    for action in rec['recommendations']:
        print(f"- {action['area']}: {action['action']}")

Optimized Budget Allocation:
LinkedIn: $4,689.25
Email: $7,955.08
Events: $12,449.41
Referrals: $24,906.26

Recommendations:

LinkedIn:

Email:

Events:

Referrals:
